## Problem description

Consider a consulting company that has three open positions: Tester, Java Developer, and Architect. 

The three top candidates (resources) for the positions are: Carlos, Joe, and Monika. 

The consulting company administered competency tests to each candidate in order to assess their ability to perform each of the jobs. 

The results of these tests are called *matching scores*. 

Assume that only one candidate can be assigned to a job, and at most one job can be assigned to a candidate.

The problem is to determine an assignment of resources and jobs such that each job is fulfilled, each resource is assigned to at most one job, and the total matching scores of the assignments is maximized.


## Mathematical optimization 

Mathematical optimization is a declarative approach where the modeler formulates an  optimization problem that captures the key features of a complex decision problem. 

The Gurobi Optimizer solves the mathematical optimization problem using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components:

* Sets
* Parameters
* Decision variables
* Constraints
* Objective function(s)

In [12]:
import gurobipy as gp
from gurobipy import GRB

## Resource Assignment Problem

### Data
The list $I$ contains the names of the three resources: Carlos, Joe, and Monika.

The list $J$ contains the names of the job positions: Tester, Java Developer, and Architect.

$i \in I$: index and set of resources. The resource $i$ belongs to the set of resources $I$.

$j \in J$: index and set of jobs. The job $j$ belongs to the set of jobs $J$.

In [13]:
# Resource and job sets
I = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

For each resource $i$ and job $j$, there is a corresponding matching score $s$. 

The matching score $s$ can only take values between 0 and 100. That is, $s_{i,j} \in [0, 100]$ for all resources $i \in I$ and jobs $j \in J$. 

We use the Gurobi Python ``multidict`` function to initialize one or more dictionaries with a single statement. The function takes a dictionary as its argument. 
The keys represent the possible combinations of resources and jobs.

In [14]:
# Matching score data
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

The following constructor creates an empty ``Model`` object “m”. 

We specify the model name by passing the string "RAP" as an argument. 

The ``Model`` object “m” holds a single optimization problem. 

It consists of a set of variables, a set of constraints, and the objective function.

In [15]:
# Declare and initialize model
m = gp.Model('RAP')

## Decision variables

To solve this assignment problem, we need to identify which resource is assigned to which job. 

We introduce a decision variable for each possible assignment of resources to jobs. 
Therefore, we have 9 decision variables.

The ``Model.addVars()`` method creates the decision variables for a ``Model`` object.
This method returns a Gurobi ``tupledict`` object that contains the newly created variables. 
We supply the ``combinations`` object as the first argument to specify the variable indices. 

The ``name`` keyword is used to specify a name for the newly created decision variables. 

By default, variables are assumed to be non-negative.

In [16]:
# Create decision variables for the RAP model
x = m.addVars(combinations, name="assign")

## Job constraints

These constraints need to ensure that each job is filled by exactly one resource.

The job constraint for the Tester position requires that resource 1 (Carlos), resource 2 (Joe), or resource 3 (Monika) is assigned to this job. 

This corresponds to the following constraint.

Constraint (Tester=1)

$$
x_{1,1} + x_{2,1} + x_{3,1} = 1
$$

Similarly, the constraints for the Java Developer and Architect positions can be defined as follows.

Constraint (Java Developer = 2)

$$
x_{1,2} + x_{2,2} + x_{3,2} = 1
$$

Constraint (Architect = 3)

$$
x_{1,3} + x_{2,3} + x_{3,3} = 1
$$

In general, the constraint for the job Tester can defined as follows.

$$
x_{1,1} + x_{2,1} + x_{3,1} = \sum_{i=1}^{3 } x_{i,1} =  \sum_{i \in I} x_{i,1} = 1
$$

All of the job constraints can be defined in a similarly succinct manner. 
For each job $j \in J$, take the summation of the decision variables over all the resources. 
We can write the corresponding job constraint as follows.

$$
\sum_{i \in I} x_{i,j} = 1
$$

The ``Model.addConstrs()`` method of the Gurobi/Python API defines the job constraints of the ``Model`` object “m”. This method returns a Gurobi ``tupledict`` object that contains the job constraints. 
The first argument of this method, "x.sum(‘*’, j)", is the sum method and defines the LHS of the jobs constraints as follows:

For each job $j$ in the set of jobs $J$, take the summation of the decision variables over all the resources. The $==$  defines an equality constraint, and the number "1" is the RHS of the constraints.
These constraints are saying that exactly one resource should be assigned to each job.
The second argument is the name of this type of constraints.

In [17]:
# Create job constraints
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

## Resource constraints

The constraints for the resources need to ensure that at most one job is assigned to each resource. 
That is, it is possible that not all the resources are assigned.

For example, we want a constraint that requires Carlos to be assigned to at most one of the jobs: either job 1 (Tester), job 2 (Java Developer ), or job 3 (Architect). We can write this constraint as follows.

Constraint (Carlos=1)

$$
x_{1, 1} + x_{1, 2} + x_{1, 3}  \leq 1.
$$

This constraint is less or equal than 1 to allow the possibility that Carlos is not assigned to any job. Similarly, the constraints for the resources Joe and Monika can be defined as follows:

Constraint (Joe=2) 

$$
x_{2, 1} + x_{2, 2} + x_{2, 3}  \leq 1.
$$

Constraint (Monika=3)

$$
x_{3, 1} + x_{3, 2} + x_{3, 3}  \leq 1.
$$

Observe that the resource constraints are defined by the rows of the following table.

The constraint for the resource Carlos can be defined as follows.

$$
x_{1, 1} + x_{1, 2} + x_{1, 3} = \sum_{j=1}^{3 } x_{1,j} = \sum_{j \in J} x_{1,j} \leq 1.
$$

Again, each of these constraints can be written in a succinct manner. For each resource $r \in R$, take the summation of the decision variables over all the jobs. We can write the corresponding resource constraint as follows.

$$
\sum_{j \in J} x_{i,j} \leq  1.
$$

The ``Model.addConstrs()`` method of the Gurobi/Python API defines the resource constraints of the ``Model`` object “m”. 
The first argument of this method, "x.sum(r, ‘*’)", is the sum method and defines the LHS of the resource constraints as follows: For each resource $r$ in the set of resources $R$, take the summation of the decision variables over all the jobs.
The $<=$  defines a less or equal constraints, and the number “1” is the RHS of the constraints.
These constraints are saying that each resource can be assigned to at most 1 job.
The second argument is the name of this type of constraints.


In [18]:
# Create resource constraints
resources = m.addConstrs((x.sum(i,'*') <= 1 for i in I), name='resource')

## Objective function

The objective function is to maximize the total matching score of the assignments that satisfy the job and resource constraints. 

For the Tester job, the matching score is $53x_{1,1}$, if resource Carlos is assigned, or $80x_{2,1}$, if resource Joe is assigned, or $53x_{3,1}$, if resource Monika is assigned.
Consequently, the matching score for the Tester job is as follows, where only one term in this summation will be nonzero.

$$
53x_{1,1} + 80x_{2,1} + 53x_{3,1}. 
$$

Similarly, the matching scores for the Java Developer and Architect jobs are defined as follows. The matching score for the Java Developer job is:

$$
27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}.
$$

The matching score for the Architect job is:

$$
13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}.
$$

The goal is to  maximize the total matching score of the assignments. Therefore, the objective function is defined as follows.

\begin{equation}
\text{Maximize} \quad (53x_{1,1} + 80x_{2,1} + 53x_{3,1}) \; +
\end{equation}

\begin{equation}
\quad (27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) \; +
\end{equation}

\begin{equation}
\quad (13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}).
\end{equation}

Each term in parenthesis in the objective function can be expressed as follows.

\begin{equation}
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{i \in I} s_{i,1} x_{i,1}.
\end{equation}

\begin{equation}
(27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) = \sum_{i \in I} s_{i,2} x_{i,2}.
\end{equation}

\begin{equation}
(13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}) = \sum_{i \in I} s_{i,3} x_{i,3}.
\end{equation}

Hence, the objective function can be concisely written as:

\begin{equation}
\text{Maximize} \quad \sum_{j \in J} \sum_{i \in I} s_{i,j} x_{i,j}.
\end{equation}

The ``Model.setObjective()`` method of the Gurobi/Python API defines the objective function of the ``Model`` object “m”. 

The objective expression is specified in the first argument of this method.

Notice that both the matching score parameters “score” and the assignment decision variables “x” are defined over the “combinations” keys. 

Therefore, we use the method “x.prod(score)” to obtain the summation of the elementwise multiplication of the "score" matrix and the "x" variable matrix.

The second argument, ``GRB.MAXIMIZE``, is the optimization "sense." In this case, we want to *maximize* the total matching scores of all assignments.

In [19]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

We use the “write()” method of the Gurobi/Python API to write the model formulation to a file named "RAP.lp".

In [20]:
# Save model for inspection
m.write('RAP.lp')

We use the “optimize( )” method of the Gurobi/Python API to solve the problem we have defined for the model object “m”.

In [21]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.930000000e+02


In [22]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


The optimal assignment is to assign:

* Carlos to the Tester job, with a matching score of 53
* Joe to the Architect job, with a matching score of 67
* Monika to the Java Developer job, with a matching score of 73.

The maximum total matching score is 193.